In [123]:
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
os.chdir('/home/witoslaw/diffTSS/trained_models/2025-07-17-17')

In [3]:
os.getcwd()

'/home/witoslaw/diffTSS/trained_models/2025-07-17-17'

In [37]:
results_dict = {}
results_dict['K562'] = pd.DataFrame()
results_dict['GM12878'] = pd.DataFrame()

for file in [file for file in os.listdir() if 'csv' in file]:
    #print(f"File: {'.'.join(file.split('.')[-5:-2])}")
    if 'K562' in file:
        results_dict['K562'] = pd.concat([results_dict['K562'], pd.read_csv(file)], ignore_index=True)
    else:
        results_dict['GM12878'] = pd.concat([results_dict['GM12878'], pd.read_csv(file)], ignore_index=True)

In [73]:
K562_h5 = h5py.File('/home/witoslaw/data/diffTSS/K562_enhancer_promoter_encoding.rna_encoding.hg38.h5')
GM12878_h5 = h5py.File('/home/witoslaw/data/diffTSS/GM12878_enhancer_promoter_encoding.rna_encoding.hg38.h5')

---
## Getting RNA signals for TSSs predicted as negative values

In [70]:
neg_pred_dict = {}
neg_pred_dict['K562'] = []
neg_pred_dict['GM12878'] = []

for key in results_dict.keys():
    results_dict[key]['error'] = abs(results_dict[key]['Pred'] - results_dict[key]['actual'])
    results_dict[key]['neg_pred'] = results_dict[key]['Pred'] < 0
    neg_pred_dict[key] = list(results_dict[key][results_dict[key]['neg_pred']].iloc[:,0])
    print(f"For {key}")
    print(f"Mean error: {np.mean(results_dict[key]['error'])}")
    print(f"Max error: {np.max(results_dict[key]['error'])}")
    print(f"Min error: {np.min(results_dict[key]['error'])}")
    print(results_dict[key][results_dict[key]['neg_pred']])

For K562
Mean error: 0.4940765429518538
Max error: 4.908513902
Min error: 1.8000000000073513e-05
              Unnamed: 0      Pred  actual  fold_idx     error  neg_pred
6        ENSG00000116183 -0.002844     0.0         1  0.002844      True
1829     ENSG00000234754 -0.004653     0.0         1  0.004653      True
2111     ENSG00000007933 -0.002311     0.0         1  0.002311      True
7317     ENSG00000134489 -0.004991     0.0         4  0.004991      True
7458     ENSG00000178522 -0.001096     0.0         4  0.001096      True
...                  ...       ...     ...       ...       ...       ...
25553    ENSG00000280809 -0.023375     0.0        11  0.023375      True
25560    ENSG00000237797 -0.003499     0.0        11  0.003499      True
25576  ENSG00000159398_1 -0.012483     0.0        11  0.012483      True
25635    ENSG00000260314 -0.008390     0.0        11  0.008390      True
25637  ENSG00000151474_2 -0.003149     0.0        11  0.003149      True

[532 rows x 6 columns]
For

In [176]:
k5_bool = [gene.decode() in neg_pred_dict['K562'] for gene in K562_h5['ensid']]
k5_rna_signals = K562_h5['rna'][:][k5_bool]

gm_bool = [gene.decode() in neg_pred_dict['GM12878'] for gene in K562_h5['ensid']]
gm_rna_signals = GM12878_h5['rna'][:][gm_bool]

In [ ]:
for signal in gm_rna_signals:
    plt.plot(signal)
    plt.show()

---
## Getting RNA signals for TSSs where error < 0.01 

In [183]:
low_error_dict = {}
low_error_dict['K562'] = []
low_error_dict['GM12878'] = []

for key in results_dict.keys():
    results_dict[key]['low_error'] = results_dict[key]['error'] < .01
    low_error_dict[key] = list(results_dict[key][results_dict[key]['low_error']].iloc[:,0])
    print(results_dict[key][results_dict[key]['low_error']])

              Unnamed: 0      Pred    actual  fold_idx     error  neg_pred  \
6        ENSG00000116183 -0.002844  0.000000         1  0.002844      True   
326    ENSG00000174606_1  2.167054  2.168792         1  0.001738     False   
335      ENSG00000271252  0.006273  0.000000         1  0.006273     False   
337    ENSG00000082512_1  1.676264  1.670793         1  0.005471     False   
534    ENSG00000120334_1  2.910146  2.917038         1  0.006892     False   
...                  ...       ...       ...       ...       ...       ...   
27629  ENSG00000107104_1  1.762994  1.771440        12  0.008447     False   
27642    ENSG00000233016  2.703006  2.694162        12  0.008844     False   
27978    ENSG00000189159  1.041013  1.034388        12  0.006625     False   
28155    ENSG00000157637  1.204168  1.198575        12  0.005593     False   
28191  ENSG00000141576_1  2.071029  2.061901        12  0.009128     False   

       low_error  
6           True  
326         True  
335   

In [1]:
k5_bool = [gene.decode() in low_error_dict['K562'] for gene in K562_h5['ensid']]
k5_rna_signals = K562_h5['rna'][:][k5_bool]

gm_bool = [gene.decode() in low_error_dict['GM12878'] for gene in K562_h5['ensid']]
gm_rna_signals = GM12878_h5['rna'][:][gm_bool]

NameError: name 'K562_h5' is not defined

In [ ]:
for signal in gm_rna_signals:
    plt.plot(signal)
    plt.show()